# Solution to the Fast Fourier Transform

## Introduction
The Fourier transform is more commonl known for its applications in digital signal processing and image processing but essentially the fourier transform enables the transformation between a polynomial and sample representation of data. The ability to represent a polynomial in  both polynomial form:

$$f(x) = a_{0}x_{0} + a_{1}x_{1}+ ...+a_{n}x_{n}$$
$$A = <a_{0}, a_{1},...,a_{n}>$$
$$X = <x_{0}, x_{1},...,x_{n}>$$

and samples form:
$$<(x,y)_{1}, (x,y)_{2}, ...,(x,y)_{n+1}$$

has its advantages. As explored in the previous task of `basic python programming` vectorized sample data has computational advantages, expecially when certain operations  are done to them like multiplication. Polynomial expression has the advantage of faster evaluation operations.

In order to represent a polynomial in its vectorized sample form, $n+1$ descrete points are required where $n$ represents the order of the polynomial. A polynomial can also be represented with its roots, but performing operations in this form is more computationally complex and difficult to conceptualize.

| Operations | Polynomials (coefficients) | Roots | Vectors(samples) |
|--------------------|:-------------------:|:----------:|-------------:|
|Evaluation|$O(n)$| $O(n)$| $O(n^{2})$|
|Addition|$O(n)$|$\infty$|$O(n)$|
|Multiplication|$O(n^{2})$|$O(n)$|$O(n)$|

for some perspective: if a set contains $n=10^{9}$ samples $O(n^{2})$ operations, if each $n$ operation takes a nanosecond, would take 30 years to complete. If there was a way to convert the polynomial in $O(n\log n)$ as to the vectorized representation, as the fast fourier transform claims to do, this same operation woud only take 30 seconds .



##Methodology

A possible strategy would be to merely evaluate $n+1$ points fom the polynomal resulting in a vector. This done naively would also result in $O(n^{2})$ operations, even after applying [Horner's method](https://en.wikipedia.org/wiki/Horner%27s_method).

The Descrete Fourier Transform exploits the relationship between polynomial coefficients and these vectorized samples. By applying a forula to the vector of coefficients $A$ the out put of the computations would be the target vector $X$.

$$x_{[k]} = \sum_{n=1}^{N-1}a_{n}e^{\frac{-2\pi}{N}nk}, k=0, 1, ...,N-1$$

Unrtunately this evaluation also amounts to $O(n^{2})$ expense. This lead to the development of the fast fourier transform. This implimentation exploits the relationship between numbers and their roots andreducing the coeffiient set to excecute the [divide and conquer algorithm](https://en.wikipedia.org/wiki/Divide_and_conquer_algorithm).  A set $W$ can be represented as a new set $W_{\frac{1}{2}}$ thats half the size of $W$ if all its constituents are the roots of the original set.  

The [fundimental theorem of algebra](https://en.wikipedia.org/wiki/Fundamental_theorem_of_algebra) explains that all polynomals have at least $p$ number of roots. This enables us to reduce our $W$ vector to $W_{\frac{1}{2}}$ as long as each variable  $w$ can be represented by its root in the original set. The vector of polynomial coefficients can autonimously be devided into 2 lists of even and odd indicies of the original vector $A$, with half its size.

$$A_{even} = \sum_{K=0}^{\frac{n}{2}-1}a_{2K}w_{\frac{n}{2}}^{K}$$

$$A_{odd} = \sum_{K=0}^{\frac{n}{2}}a_{2K+1}w_{\frac{n}{2}}^K$$

where $w=e^{\frac{2\pi i}{n}}$.
These can then be combined to calculate the target valuer:


$$x = A_{even}(w_{\frac{n}{2]}^{2}) +x * A_{odd}(w_{\frac{n}{2}}^{2})   $$

we now have the building blocks to apply a recursive method to our devide and conquer strategy. It also indicates one condition the data has to have: the set of coefficients has to be a power of 2.

Since we are dealing with the roots of unity, in order or the set to consecutively half with each iteration, we can apply the followig formula:

$$ (w_{n}^{k+\frac{n}{2}})^{2} = (w_{n}^{k})^{2} $$

After each iteration we essentially calculate $w$ values that with parametes that can be used for more that one $a$, this enables us to compute the fourier transform recursively on halved lists, much like the spider diagram

![alt text](https://naiadseye.files.wordpress.com/2013/10/butterfly-algorithm-diagram.jpg)

In this implimentation the a random list of values, representing polynomial coefficients. We will compute the Vandermande matrix, and compare our calculations to the matrix generated by the Numpy implimentation of the FFT

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import math

In [0]:
import numpy as np
def DFT(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = np.exp(-2j * np.pi * k * n / N)
    return np.dot(M, x)

In [0]:
def FFT(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    
    if N % 2 > 0:
        raise ValueError("size of x must be a power of 2")
    elif N <= 2:  # this cutoff should be optimized
        return DFT(x)
    else:
        X_even = FFT(x[::2])
        X_odd = FFT(x[1::2])
        factor = np.exp(-2j * np.pi * np.arange(N) / N)
        return np.concatenate([X_even + factor[:N // 2] * X_odd,
                               X_even + factor[N // 2:] * X_odd])

In [4]:
x = np.random.random(1024)
np.allclose(FFT(x), np.fft.fft(x))

True